In [112]:
import pandas as pd
import json

df_csv = pd.read_csv('UNSPSC_English.csv')
df_xlsx = pd.read_excel('Avalara_goods_and_services.xlsx')

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3194: DtypeWarning: Columns (1,2,5,8,9,12,15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [113]:
# Convert row to header and remove the row
df_csv = df_csv.rename(columns=df_csv.iloc[8]).loc[9:100]
df_csv = df_csv.drop(columns = ['Version', 'Key', 'Column1', 'Synonym', 'Acronym'])
df_csv

,Segment,Segment Title,Segment Definition,Family,Family Title,Family Definition,Class,Class Title,Class Definition,Commodity,Commodity Title,Definition
9,10000000,Live Plant and Animal Material and Accessories...,"This segment includes live, wild and domestica...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,10000000,Live Plant and Animal Material and Accessories...,"This segment includes live, wild and domestica...",10100000,Live animals,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,10000000,Live Plant and Animal Material and Accessories...,"This segment includes live, wild and domestica...",10100000,Live animals,NaN,10101500,Livestock,NaN,NaN,NaN,NaN
12,10000000,Live Plant and Animal Material and Accessories...,"This segment includes live, wild and domestica...",10100000,Live animals,NaN,10101500,Livestock,NaN,10101501,Cats,NaN
13,10000000,Live Plant and Animal Material and Accessories...,"This segment includes live, wild and domestica...",10100000,Live animals,NaN,10101500,Livestock,NaN,10101502,Dogs,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
96,10000000,Live Plant and Animal Material and Accessories...,"This segment includes live, wild and domestica...",10100000,Live animals,NaN,10102000,Wild animals,NaN,10102002,Live foxes,NaN
97,10000000,Live Plant and Animal Material and Accessories...,"This segment includes live, wild and domestica...",10100000,Live animals,NaN,10102000,Wild animals,NaN,10102003,Live bothrops pit viper snake,A venomous pit viper snake spanning 32 species...
98,10000000,Live Plant and Animal Material and Accessories...,"This segment includes live, wild and domestica...",10100000,Live animals,NaN,10102000,Wild animals,NaN,10102004,Live chironius or vine snake,A type of tree-dwelling snake spanning 13 spec...
99,10000000,Live Plant and Animal Material and Accessories...,"This segment includes live, wild and domestica...",10100000,Live animals,NaN,10102000,Wild animals,NaN,10102005,Live clelia or mussurana snake,A type of snake spanning 10 species whose prey...


In [114]:
df_xlsx = df_xlsx.rename(columns={'Unnamed: 0':'AvaTax System Tax Code', 'AvaTax System Tax Codes':'AvaTax System Tax Code Description',
                                 'Unnamed: 2':'Additional AvaTax System Tax Code Information'})
df_xlsx = df_xlsx[2:50]
df_xlsx

,AvaTax System Tax Code,AvaTax System Tax Code Description,Additional AvaTax System Tax Code Information
2,D0000000,Digital goods,Digital goods are generally viewed as download...
3,D9999999,Temporary unmapped digital goods sku - taxable...,A digital good that has not been mapped to a s...
4,DA010000,Digital goods: advertising material (digital o...,"Tangible personal property (i.e. designs, artw..."
5,DA030000,Digital goods: art (digital only -- no physica...,Canned artwork that is transferred to the cust...
6,DA040000,Digital goods: audio (digital only -- no physi...,Audio (i.e. music or spoken word) that is tran...
7,DA040100,Radio subscriptions,Subscriptions to satellite radio services.
8,DA050000,Digital audio works,NaN
9,DA051011,Digital audio works sold to an end user with r...,NaN
10,DA051012,Digital audio works (with rights conditioned o...,NaN
11,DA051013,Digital audio works sold to users other than t...,NaN


In [115]:
# import nltk and download resources
import nltk
import ssl
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /Users/mikec/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/mikec/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [116]:
# parse the stop words and special characters out
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english'))
stop_words.add(':')
stop_words.add('--')
stop_words.add('-')
stop_words.add('(')
stop_words.add(')')
stop_words.add('/')
stop_words.add('*')
stop_words.add(';')

column_name = 'AvaTax System Tax Code Description'

for index, description in enumerate(df_xlsx[column_name], 2):
    word_tokens = word_tokenize(description)

    filtered_sentence = []
    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence.append(w)

    # remove empty strings
    while "" in filtered_sentence:
        filtered_sentence.remove("")

    # updating record in AvaTax System Tax Code Description
    df_xlsx.at[index, column_name]= ','.join(filtered_sentence)

df_xlsx

,AvaTax System Tax Code,AvaTax System Tax Code Description,Additional AvaTax System Tax Code Information
2,D0000000,"Digital,goods",Digital goods are generally viewed as download...
3,D9999999,"Temporary,unmapped,digital,goods,sku,taxable,d...",A digital good that has not been mapped to a s...
4,DA010000,"Digital,goods,advertising,material,digital,phy...","Tangible personal property (i.e. designs, artw..."
5,DA030000,"Digital,goods,art,digital,physical,media",Canned artwork that is transferred to the cust...
6,DA040000,"Digital,goods,audio,digital,physical,media",Audio (i.e. music or spoken word) that is tran...
7,DA040100,"Radio,subscriptions",Subscriptions to satellite radio services.
8,DA050000,"Digital,audio,works",NaN
9,DA051011,"Digital,audio,works,sold,end,user,rights,perma...",NaN
10,DA051012,"Digital,audio,works,rights,conditioned,continu...",NaN
11,DA051013,"Digital,audio,works,sold,users,end,user",NaN


In [117]:
tc_hash_map = {}

for tax_code in df_xlsx['AvaTax System Tax Code']:
    level_1 = tax_code[0:2]
    level_2 = tax_code[2:4]
    level_3 = tax_code[4:8]
    string_placeholder = []

    if level_1 not in tc_hash_map:
        tc_hash_map[level_1] = {
            level_2: {level_3: string_placeholder}
        }
    elif level_2 not in tc_hash_map[level_1]:
        tc_hash_map[level_1][level_2] = {level_3: string_placeholder}
    elif level_3 not in tc_hash_map[level_1][level_2]:
        tc_hash_map[level_1][level_2][level_3] = string_placeholder

print(json.dumps(tc_hash_map, sort_keys=True, indent=2))

{
  "D0": {
    "00": {
      "0000": []
    }
  },
  "D9": {
    "99": {
      "9999": []
    }
  },
  "DA": {
    "01": {
      "0000": []
    },
    "03": {
      "0000": []
    },
    "04": {
      "0000": [],
      "0100": []
    },
    "05": {
      "0000": [],
      "1011": [],
      "1012": [],
      "1013": [],
      "9399": []
    }
  },
  "DB": {
    "01": {
      "0000": []
    },
    "02": {
      "0000": []
    },
    "03": {
      "1000": [],
      "1013": [],
      "1014": [],
      "1015": [],
      "1016": []
    }
  },
  "DC": {
    "01": {
      "0000": [],
      "0100": [],
      "0200": [],
      "0300": [],
      "0400": [],
      "0500": [],
      "0600": [],
      "1000": []
    },
    "02": {
      "0000": [],
      "0100": [],
      "0200": [],
      "0300": [],
      "0400": [],
      "0402": [],
      "0500": [],
      "0501": [],
      "0502": [],
      "0503": [],
      "0600": []
    },
    "06": {
      "0000": []
    },
    "07": {
      "0000": []
   